# Workload Insights Notebook

This notebook analyzes the denormalized representation of query workload. This denormalized dataset contains the information from applications, queries, plans, and runtime metrics. The plans were also annotated with signatures to identify subexpressions. We hope that the analysis in the notebook helps you in making data-driven decisions.


## Features
1. <a href=#sample>Sample Rows</a>
2. <a href=#qcount>Query and Operator Counts</a>
3. <a href=#opfreq>Operator Frequencies</a>
4. <a href=#cquery>Overlapping Queries</a>
5. <a href=#csubop>Common Subexpressions Per Operator</a>
6. <a href=#selviews>Selected Views</a>
7. <a href=#ss>SparkCruise Savings</a>
8. <a href=#filterstat>Filter Selectivity</a>
9. <a href=#exstat>Exchange Operator Statistics</a>
10. <a href=#recjobs>Recurring Jobs</a>

## Setup

In [ ]:
logicalExps = spark.read.format("csv").option("sep", "|").option("schema","AppID: string (nullable = true), ClusterName: string (nullable = true), Subscription: string (nullable = true), QueryID: integer (nullable = true), AppQueryID: string (nullable = true), OperatorName: string (nullable = true), TreeLevel: integer (nullable = true), ChildCount: integer (nullable = true), StrictSignature: integer (nullable = true), NonStrictSignature: integer (nullable = true), Parameters: string (nullable = true)").option("header", "true").load("/peregrine/views/logical_ir.csv")
logicalExps.createOrReplaceTempView("LogicalExps")
physicalExps = spark.read.format("csv").option("sep", "|").option("schema","AppID: string (nullable = true), ClusterName: string (nullable = true), Subscription: string (nullable = true), QueryID: integer (nullable = true), AppQueryID: string (nullable = true), OperatorName: string (nullable = true), TreeLevel: integer (nullable = true), ChildCount: integer (nullable = true), StrictSignature: integer (nullable = true), NonStrictSignature: integer (nullable = true), Parameters: string (nullable = true)").option("header", "true").load("/peregrine/views/physical_ir.csv")
physicalExps.createOrReplaceTempView("PhysicalExps")
analysisExps = spark.sql("SELECT * FROM LogicalExps WHERE ChildCount > 0")
analysisExps.createOrReplaceTempView("AnalysisExps")
repeatSubexps = spark.sql("SELECT StrictSignature FROM AnalysisExps GROUP BY StrictSignature HAVING COUNT(DISTINCT AppQueryID) > 1")
repeatSubexps.createOrReplaceTempView("RepeatSubexps")

## Sample Rows <a name='sample' /> 
Example of records in intermediate representation.


In [ ]:
%%sql
SELECT AppID, AppName, AppStartTime, QueryID, QueryWallClockTime, OperatorName, LogicalName, StrictSignature, PRowCount, PExclusiveTime
FROM PhysicalExps 
WHERE LENGTH(StrictSignature) > 0 AND PExclusiveTime > 0 
ORDER BY rand() 
LIMIT 10

## Query and Operator Count <a name='qcount' /> 

Number of queries and operators in workload.

In [ ]:
%%sql
SELECT COUNT(DISTINCT AppQueryID) AS QueryCount, COUNT(*) AS OperatorCount, COUNT(DISTINCT OperatorName) AS DistinctOperators
FROM PhysicalExps

## Operator Frequency <a name='opfreq' /> 
Frequency of logical and physical operators in workload.


In [ ]:
%%sql
SELECT OperatorName, COUNT(*) AS Frequency
FROM LogicalExps
GROUP BY OperatorName
ORDER BY Frequency DESC

Frequency of physical operators in workload.


In [ ]:
%%sql
SELECT OperatorName, COUNT(*) AS Frequency
FROM PhysicalExps
GROUP BY OperatorName
ORDER BY Frequency DESC

### Overlapping queries <a name='cquery' /> 
Queries with overlapping computations.


In [ ]:
queryCountQuery = """
SELECT COUNT(DISTINCT AppQueryID) AS QueryCount 
FROM AnalysisExps"""
queryCount = spark.sql(queryCountQuery)
queryCount.createOrReplaceTempView("QueryCount")

queriesWithRepeatQuery = """
SELECT COUNT(DISTINCT AppQueryID) AS QueriesWithOneOverlap
FROM AnalysisExps
WHERE StrictSignature IN 
( SELECT StrictSignature
FROM RepeatSubexps )"""
queriesWithRepeat = spark.sql(queriesWithRepeatQuery)
queriesWithRepeat.createOrReplaceTempView("QueriesWithRepeat")

queriesWithTwoRepeatsQuery = """
SELECT COUNT(*) AS QueriesWithTwoOverlaps FROM (
SELECT AppQueryID, COUNT(*) AS Repeats 
FROM AnalysisExps
WHERE StrictSignature IN 
( SELECT StrictSignature
FROM RepeatSubexps )
GROUP BY AppQueryID
HAVING Repeats > 1
ORDER BY Repeats DESC)"""
queriesWithTwoRepeats = spark.sql(queriesWithTwoRepeatsQuery)
queriesWithTwoRepeats.createOrReplaceTempView("QueriesWithTwoRepeats")

In [ ]:
%%sql
SELECT *, CAST((QueriesWithOneOverlap/QueryCount)*100 AS Decimal(38,2)) AS OverlapPercent 
FROM QueryCount AS R1, QueriesWithRepeat AS R2, QueriesWithTwoRepeats AS R3

## Overlapping Computations <a name='csubop' /> 

Overlapping computations per operator.


In [ ]:
opFreqQuery = """
SELECT OperatorName, COUNT(*) AS Total
FROM AnalysisExps
GROUP BY OperatorName
ORDER BY Total DESC"""
opFreq = spark.sql(opFreqQuery)
opFreq.createOrReplaceTempView("OpFreq")

opRepeatSubexpQuery = """
SELECT OperatorName, COUNT(*) AS Repeats
FROM AnalysisExps
WHERE StrictSignature IN 
( SELECT StrictSignature
FROM RepeatSubexps )
GROUP BY OperatorName
ORDER BY Repeats DESC"""
opRepeatSubexp = spark.sql(opRepeatSubexpQuery)
opRepeatSubexp.createOrReplaceTempView("OpRepeatSubexp")

opDistinctRepeatQuery = """
SELECT OperatorName, COUNT(*) AS DistinctRepeats
FROM ( SELECT DISTINCT OperatorName, StrictSignature 
FROM AnalysisExps
WHERE StrictSignature IN 
( SELECT StrictSignature
FROM RepeatSubexps ))
GROUP BY OperatorName
ORDER BY DistinctRepeats DESC"""
opDistinctRepeat = spark.sql(opDistinctRepeatQuery)
opDistinctRepeat.createOrReplaceTempView("OpDistinctRepeat")

In [ ]:
%%sql
SELECT R1.OperatorName, Total, Repeats, DistinctRepeats, CAST(Repeats/DistinctRepeats AS Decimal(38,2)) AS AvgRepFrequency, CAST((Repeats/Total)*100 AS Decimal(38,2)) AS RepeatPercent  
FROM OpFreq AS R1, OpRepeatSubexp AS R2, OpDistinctRepeat AS R3
WHERE R1.OperatorName = R2.OperatorName AND R2.OperatorName = R3.OperatorName
ORDER BY RepeatPercent DESC

## Selected Views <a name='selviews' /> 
Per operator summary of selected views.

In [ ]:
selViews = spark.read.format("csv").option("sep", "|").option("schema","AppID: string (nullable = true), ClusterName: string (nullable = true), Subscription: string (nullable = true), QueryID: integer (nullable = true), AppQueryID: string (nullable = true), OperatorName: string (nullable = true), TreeLevel: integer (nullable = true), ChildCount: integer (nullable = true), StrictSignature: integer (nullable = true), NonStrictSignature: integer (nullable = true), Parameters: string (nullable = true)").option("header", "true").load("/peregrine/views/views.csv")
selViews.createOrReplaceTempView("SelViews")
views = spark.sql("SELECT DISTINCT StrictSignature FROM SelViews")
views.createOrReplaceTempView("Views")

In [ ]:
distinctViewsQuery = """
SELECT P.LogicalName AS LogicalName, COUNT(DISTINCT P.StrictSignature) AS ViewCount
FROM Views V, PhysicalExps P
WHERE V.StrictSignature = P.StrictSignature 
GROUP BY P.LogicalName
ORDER BY ViewCount DESC"""
distinctViews = spark.sql(distinctViewsQuery)
distinctViews.createOrReplaceTempView("DistinctViews")

viewSubexprsOpsQuery = """
SELECT P.LogicalName AS LogicalName, COUNT(*) AS ViewRepeats
FROM Views V, PhysicalExps P
WHERE V.StrictSignature = P.StrictSignature 
GROUP BY P.LogicalName
ORDER BY ViewRepeats DESC"""
viewSubexprsOps = spark.sql(viewSubexprsOpsQuery)
viewSubexprsOps.createOrReplaceTempView("ViewSubexprsOps")

In [ ]:
%%sql
SELECT X.LogicalName, ViewCount, ViewRepeats
FROM DistinctViews AS X, ViewSubexprsOps AS Y
WHERE X.LogicalName = Y.LogicalName

## SparkCruise Savings <a name='ss' /> 
Potential savings per view.


In [ ]:
%%sql
SELECT LogicalName, NumOccurrences, AvgSerialTime_ms, AvgRowCount, AvgRowLength_bytes
FROM (
SELECT P.LogicalName AS LogicalName, P.StrictSignature AS Id, COUNT(*) AS NumOccurrences, AVG(PSerialTime) AS AvgSerialTime_ms, AVG(PRowCount) AS AvgRowCount, AVG(AvgRowLength) AS AvgRowLength_bytes
FROM Views V, PhysicalExps P
WHERE V.StrictSignature = P.StrictSignature
GROUP BY P.LogicalName, P.StrictSignature)

## Filter Selectivity <a name='filterstat' /> 
Selectivity of filters.


In [ ]:
%%sql -o filterSel

SELECT F.AppQueryID, F.OperatorName AS FilterOp, F.RowCount AS PassCount, T.OperatorName AS ScanOp, T.RowCount AS TotalCount, CAST(F.RowCount/T.RowCount AS Decimal(38, 5)) AS FilterSel
FROM PhysicalExps F, PhysicalExps T 
WHERE F.AppQueryID = T.AppQueryID AND
F.OperatorName = 'FilterExec' AND 
T.ParentID = F.OperatorID AND
T.RowCount > 0 AND 
T.ChildCount = 0
ORDER BY FilterSel

In [ ]:
%%local
%matplotlib inline
import matplotlib.pyplot as plt

# CDF
filterSel['pdf'] = filterSel['FilterSel']/sum(filterSel['FilterSel'])
filterSel['ecdf'] = (filterSel['pdf'].cumsum())
ax = filterSel.plot(x = 'FilterSel', y = 'ecdf', grid = True)
ax.set_xlabel("Filter Selectivity")
ax.set_ylabel("CDF")
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)

## Exchange <a name='exstat' /> 
How many rows are shuffled in real-world Spark workloads?

In [ ]:
def getBucket(rowCount):
    rows = int(rowCount)
    if rows <=1:
        return "a. 0-1"
    elif (rows > 1 and rows <= 1e3):
        return "b. 1-1k"
    elif (rows > 1e3 and rows <= 1e5):
        return "c. 1k-100k"
    elif (rows > 1e5 and rows <= 1e6):
        return "d. 100k-1M"
    elif (rows > 1e6 and rows <= 1e8):
        return "e. 1M-100M"
    elif (rows > 1e8 and rows <=  1e9):
        return "f. 100M-1B"
    else:
        return "g. >1B"

spark.udf.register("getBucket", getBucket)
exBucketQuery = """ SELECT Bucket AS RowCountBucket, CAST(AVG(MB) AS Decimal(38,2)) AS AvgDataSizeInMB, CAST(MAX(MB) AS Decimal(38,2)) AS MaxDataSizeInMB, COUNT(*) AS Count
FROM (
SELECT getBucket(PRowCount) AS Bucket, Bytes/(1024.0*1024) AS MB
FROM PhysicalExps
WHERE PRowCount > 0 AND Bytes > 0 AND OperatorName LIKE '%ShuffleExchangeExec%')
GROUP BY Bucket
ORDER BY RowCountBucket
"""
exBucket = spark.sql(exBucketQuery)
exBucket.show(100, False)
exBucket.createOrReplaceTempView("ExBucket")

In [ ]:
%%sql -o exBuckets
SELECT * 
FROM ExBucket

In [ ]:
%%local
%matplotlib inline
import matplotlib.pyplot as plt

exBuckets['RCBucketLabels'] = exBuckets['RowCountBucket'].str[3:]

ax = exBuckets.plot.bar(x='RCBucketLabels', y='Count', rot=0)
ax.set_xlabel("Number of rows in exchange")
ax.set_ylabel("Frequency")

## Recurring jobs <a name='recjobs' /> 
Recurring jobs share same non-strict signature at the root level. To capture temporal patterns, we can take the intersection of the result set for a few consecutive days and then split the intesection set in hourly, daily repeat patterns based on the AppSubmitTime value.

In [ ]:
%%sql
SELECT Subscription, NonStrictSignature, COUNT(*) AS Count
FROM AnalysisExps
WHERE TreeLevel = 0 AND LENGTH(NonStrictSignature) > 0
GROUP BY Subscription, NonStrictSignature
HAVING COUNT(*) > 1
ORDER BY Count DESC